# Demo of the developed camera calibration features

In [ ]:
import numpy as np
import cv2 as cv
import matplotlib.pyplot as plt

In [ ]:
from scripts.utils import imshow, plot_world_point

In [ ]:
maracana1 = cv.imread("maracana1.jpg")
imshow(maracana1, "Original image")

Now, let's choose a couple of correspondences between world points and image points.

In [ ]:
# Homogeneous component
s = 1

# Define world points
world_points = np.array([
    [12.82,0,  0],
    [23.82,0,  0],
    [0,    0,  0],
    [0,    0,  2.44],
    [7.32, 0,  2.44],
    [12.82,5.5,  0],
    [-16.5,0,  0],
    [7.32,0,  0],
    [-5.5, 0, 0],
    [-5.5, 5.5, 0],
    [-16.5, 16.5, 0]
])

# Define image points
img_points = np.array([
    [183.6, 126.8],
    [227.7, 104.9],
    [124, 156.3],
    [122, 125.3],
    [158.4, 109.8],
    [241.2, 130.6],
    [30.4, 203.8],
    [159.3, 138.4],
    [94.9, 171.8],
    [160.8, 175.2],
    [249, 220.8]
])

In [ ]:
for point in img_points:
    color = (0,255,0) if np.array_equal(point,[124, 156.3]) else (0,0,255)
    point = tuple(map(int, point))
    cv.circle(maracana1, center=point, radius=2, color=color, thickness=-1)

imshow(maracana1, "Points chosen for calibration")

Corners are preferable for mitigating calibration errors, since they provide a clear sense of position on a world plane.

Origin is highlighted in green.

Now it's time to test our DLT function.

In [ ]:
from scripts.dlt import DLT

In [ ]:
m = DLT(world_points, img_points)

In [ ]:
def find_world_point(u, v, m):
    # Extract camera matrix rows
    h11, h12, h14 = m[0, 0], m[0, 1], m[0, 3]
    h21, h22, h24 = m[1, 0], m[1, 1], m[1, 3]
    h31, h32, h34 = m[2, 0], m[2, 1], m[2, 3]

    # Formulate the equations
    A = np.array([
        [h11 - u * h31, h12 - u * h32],
        [h21 - v * h31, h22 - v * h32]
    ])
    b = np.array([
        u * h34 - h14,
        v * h34 - h24
    ])

    # Solve for x_w and y_w
    x_w, y_w = np.linalg.solve(A, b)
    return x_w, y_w

In [ ]:
u = float(input("Please insert X image coordinate for the player's feet: "))
v = float(input("Please insert Y image coordinate for the player's feet: "))
z = float(input("Please insert the player's height in meters: "))
x_w, y_w = find_world_point(u,v,m)

In [ ]:
players_feet = plot_world_point(x_w, y_w, 0, m)
players_head = plot_world_point(x_w, y_w, z, m)
# Reset image back to original state
maracana1 = cv.imread("maracana1.jpg")
cv.line(maracana1, players_feet, players_head,color=(255, 0, 0), thickness=1)
imshow(maracana1, "Image with one player positioned.")

In [ ]:
u = float(input("Please insert X image coordinate for the goalkeeper's feet: "))
v = float(input("Please insert Y image coordinate for the goalkeeper's feet: "))
z = float(input("Please insert the goalkeeper's height in meters: "))
x_w, y_w = find_world_point(u,v,m)

In [ ]:
players_feet = plot_world_point(x_w, y_w, 0, m)
players_head = plot_world_point(x_w, y_w, z, m)
cv.line(maracana1, players_feet, players_head,color=(255, 255, 0), thickness=1)
imshow(maracana1, "Image with goalkeeper positioned.")

Considering that the goal's height is 2.44 meters, the goalkeeper represented by the plotted yellow line seems to have height fairly close to 1.8 meters as specified.

## Let's dive into the second picture to try and plot an offside line

In [ ]:
maracana2 = cv.imread('maracana2.jpg')
imshow(maracana2, "Original image")

In [ ]:
# Define world points
world_points = np.array([
[-23.82, 16.5],
[16.5,16.5],
[5.5,5.5],
[-12.82,5.5],
[16.5,0],
[5.5,0],
[0,0],
[-3.66,11]
])

# Define image points
img_points = np.array([
[266, 237],
[268,61],
[474,99],
[509,177],
[550,61],
[575,99],
[590,118],
[377,134]
])

In [ ]:
maracana2 = cv.imread('maracana2.jpg')
for i, point in enumerate(img_points):
    color = (255,0,0) if np.array_equal(point,[590, 118]) else (255,255,0)
    point = tuple(map(int, point))
    cv.circle(maracana2, center=point, radius=3, color=color, thickness=-1)
imshow(maracana2, title="Points chosen for calibration")

Points chosen for calibration are plotted in yellow, origin is highlighted in red.

Now it's time to test our Homography function.

In [ ]:
def homography(world_points, img_points):
    A = []
    for i in range(len(world_points)):
        x, y = world_points[i]
        u, v = img_points[i]
        A.append([x,y,1,0,0,0,-u*x,-u*y,-u])
        A.append([0,0,0,x,y,1,-v*x,-v*y,-v])

    A = np.asarray(A)

    U, S, Vh = np.linalg.svd(A, full_matrices=False)
    m = Vh[-1,:]/Vh[-1,-1]
    return m.reshape(3,3)

In [ ]:
h = homography(world_points, img_points)

In [ ]:
def find_world_point_on_plane(u, v, m):
    # Construct the image point in homogeneous coordinates
    img_point = np.array([u, v, 1.0])

    # Compute the world point in homogeneous coordinates
    world_point_homogeneous = np.linalg.inv(m) @ img_point

    # Normalize the world point
    x_w = world_point_homogeneous[0] / world_point_homogeneous[2]
    y_w = world_point_homogeneous[1] / world_point_homogeneous[2]
    return x_w, y_w

In [ ]:
u = float(input("Please insert X image coordinate of last defender's foot: "))
v = float(input("Please insert Y image coordinate of last defender's foot: "))

x_w2, y_w2 = find_world_point_on_plane(u,v,h)

Now we must use the estimated field width so that it is possible to plot a line that's parallel to the end

In [ ]:
maracana2 = cv.imread('maracana2.jpg')
corner = plot_plane_point_2D(30, y_w2, h)
corner_2 = plot_plane_point_2D(-37.32, y_w2, h)

In [ ]:
maracana2 = cv.imread('maracana2.jpg')
cv.line(maracana2, corner, corner_2,color=(255, 0, 0), thickness=2)
imshow(maracana2, "Offside line plotted in red.")